<a href="https://colab.research.google.com/github/nhatduong01/SQL_on_BigQuery/blob/main/Basis/StackOverFlow_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Working with the `stackoverflow` dataset.

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [2]:
# Set up the Project Environment
import os
os.environ["GCLOUD_PROJECT"] = "BigQuerySQLExercises"

In [3]:
from google.cloud import bigquery
import pandas as pd


client = bigquery.Client()

data_ref = client.dataset(dataset_id= 'stackoverflow',
                          project = "bigquery-public-data")
dataset = client.get_dataset(data_ref)
tables = list(client.list_tables(dataset))
# See all the table ids in the table_list
for table in tables:
  print(table.table_id)

badges
comments
post_history
post_links
posts_answers
posts_moderator_nomination
posts_orphaned_tag_wiki
posts_privilege_wiki
posts_questions
posts_tag_wiki
posts_tag_wiki_excerpt
posts_wiki_placeholder
stackoverflow_posts
tags
users
votes


In [ ]:
# get the post_questions table
table_ref = dataset.table('posts_questions')
questions_table = client.get_table(table_ref)
# print few lines
client.list_rows(questions_table, max_results= 5).to_dataframe()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,67757982,Carry select adder pipelining in vhdl,<p>Please help me how can i pipeline this desi...,None,0,0,None,2021-05-30 05:26:15.923000+00:00,None,2021-05-30 05:26:15.923000+00:00,None,None,None,None,15758793,None,1,0,vhdl,1
1,67472359,filebeat registry file shows offset to 0,"<p>We have configured filebeat in the server, ...",None,0,0,None,2021-05-10 14:31:52.090000+00:00,None,2021-05-10 14:31:52.090000+00:00,None,None,None,None,11549576,None,1,0,offset|filebeat,2
2,67485809,Multilayer Graph/Multilayer Network and Graph ...,<p>I'm new to building graphs and GCNs and I w...,None,0,0,None,2021-05-11 11:26:22.747000+00:00,None,2021-05-11 11:26:22.747000+00:00,None,None,None,None,13916429,None,1,0,graph|neural-network|multi-layer,2
3,67514172,Creating a boxed outline in theme in wordpress,<p>Want to remove the green highlighted area a...,None,0,0,None,2021-05-13 05:11:07.957000+00:00,None,2021-05-13 05:11:07.957000+00:00,None,None,None,None,15319352,None,1,-1,frontend|wordpress-theming|themes|elementor,2
4,67570205,Regarding the Reshaping the data and Improving...,<p>1.How could I resolve the layer incompatobi...,None,0,0,None,2021-05-17 13:00:28.623000+00:00,None,2021-05-17 13:00:28.623000+00:00,None,None,None,None,15064017,None,1,0,lstm,2


In [ ]:
# get the post_answer table
table_ref = dataset.table('posts_answers')
answers_table = client.get_table(table_ref)
# print few lines
client.list_rows(answers_table, max_results= 5).to_dataframe()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,67154495,None,<p>You can get two arrays of characters from t...,None,None,0,None,2021-04-18 23:54:41.763000+00:00,None,2021-04-19 00:01:50.707000+00:00,2021-04-19 00:01:50.707000+00:00,user15675591,None,user15675591,None,64050791,2,0,None,None
1,67154892,None,<p>To check if <em>two strings are anagrams</e...,None,None,0,None,2021-04-19 01:18:10.110000+00:00,None,2021-04-19 01:18:10.110000+00:00,NaT,None,None,user15675591,None,63352574,2,0,None,None
2,67155332,None,<p>You can first get arrays of characters from...,None,None,0,None,2021-04-19 02:44:10.033000+00:00,None,2021-04-19 02:59:37.867000+00:00,2021-04-19 02:59:37.867000+00:00,user15675591,None,user15675591,None,60867630,2,0,None,None
3,67156330,None,<p>You can use <em>Java 8 Stream</em>: <a href...,None,None,0,None,2021-04-19 05:21:34.760000+00:00,None,2021-04-25 15:42:47.580000+00:00,2021-04-25 15:42:47.580000+00:00,user15675591,None,user15675591,None,66065890,2,0,None,None
4,67156874,None,<p>As others have already mentioned that &quot...,None,None,0,None,2021-04-19 06:22:17.893000+00:00,None,2021-04-19 06:22:17.893000+00:00,NaT,None,None,user10064176,None,57751417,2,0,None,None


### 1) Choose the right answer:
we will write a query that selects the `id`, `title` and `owner_user_id` columns from the `posts_questions` table. 
- Restrict the results to rows that contain the word "bigquery" in the `tags` column. 
- Include rows where there is other text in addition to the word "bigquery" (e.g., if a row has a tag "bigquery-sql", the results should include that too).

In [ ]:
query1 = """
         SELECT id, title, owner_user_id
         FROM `bigquery-public-data.stackoverflow.posts_questions`
         WHERE tags LIKE '%bigquery%'"""
query_job1 = client.query(query1, project = 'bigquerysqlexercises')
bigquery_questions = query_job1.to_dataframe()
bigquery_questions

,id,title,owner_user_id
0,67594503,Error loading location southamerica-west1: Big...,13938163.0
1,67757483,Error using bq mk command for creating empty t...,14128359.0
2,67638357,Join Google Ads (Campaign Manager) data with G...,13450264.0
3,67752915,BigQueryInsertJobOperator missing required aut...,14444816.0
4,67529732,BigQuery's Data Transfer Service - list of loa...,10144000.0
...,...,...,...
17096,52169443,Emoji crashed when uploading to Big Query,NaN
17097,52385444,"SQL Create ""Admin"" right inside SQL Query ( CA...",NaN
17098,66039840,How to join tables to result in final table th...,NaN
17099,59197388,Is there a way to remove characters in an arra...,NaN


### 2) Join questions and answers:

We will write a query that returns the `id`, `body` and `owner_user_id` columns from the `posts_answers` table for answers to "bigquery"-related questions. 



In [ ]:
query2 = """
            SELECT answer.id, answer.body, answer.owner_user_id
            FROM `bigquery-public-data.stackoverflow.posts_answers` AS answer
            INNER JOIN `bigquery-public-data.stackoverflow.posts_questions`
            AS question ON question.id = answer.parent_id
            WHERE question.tags LIKE '%bigquery%'"""
query_job2 = client.query(query2, project ='bigquerysqlexercises')

answers_table = query_job2.to_dataframe()
answers_table

,id,body,owner_user_id
0,58759989,<p>Cloud Composer (Apache Airflow) is used to ...,3226630.0
1,58764625,<p>I think this use case can be implemeted usi...,1073129.0
2,58794254,<p>I think this will helpful.</p>\n\n<p>Two ba...,5828160.0
3,58808009,<p>I do not think that delete the table fix th...,10409068.0
4,58808090,<p>This probably depends on what you are optim...,7742981.0
...,...,...,...
21544,23682318,<p>Write only authorization (specifically for ...,1366527.0
21545,24073408,<p>If you're trying to download a large table ...,1366527.0
21546,23705397,<p>Steps 1-4 can all be done in a single atomi...,1366527.0
21547,23945437,<p>This has now been fixed. You should no long...,1366527.0


### 3) `bigquery` expertise
We will write a new query that has a single row for each user who answered at least one question with a tag that includes the string "bigquery". The results should have two columns:
- `user_id` - contains the `owner_user_id` column from the `posts_answers` table
- `number_of_answers` - contains the number of answers the user has written to "bigquery"-related questions

In [6]:
query3 = """
         SELECT answer.owner_user_id AS user_id, COUNT(1) AS number_of_answers
         FROM `bigquery-public-data.stackoverflow.posts_answers` AS answer
         INNER JOIN `bigquery-public-data.stackoverflow.posts_questions` AS
         question ON question.id = answer.parent_id
         WHERE question.tags LIKE '%bigquery%'
         GROUP BY user_id"""
query_job3 = client.query(query3, project = 'bigquerysqlexercises')
bigquery_expert = query_job3.to_dataframe()
bigquery_expert

,user_id,number_of_answers
0,6823528.0,25
1,7775111.0,2
2,7676314.0,1
3,6288114.0,1
4,2790312.0,6
...,...,...
4781,1048369.0,2
4782,1902975.0,1
4783,4890357.0,1
4784,871134.0,1


$\rightarrow$ With general ideas like this, we could write a function a return the most talented `user_id` for a specific field.